# 5.1 自動化與排程

我們學會了如何開發網頁爬蟲，接下來我們只要每天執行我們寫好的程式，就能夠享受程式化帶來的便利性：讓程式自動將個股的交易資料截取下來，並寫入 Excel 内。我們只需要在執行完畢時檢查顯示在 Excel 工作表上的結果即可。

但是過了不久之後，我們就會發現要要透過手動的方式特定的時間點執行我們的程式是很困難的，原因可能是忘記、或是在該執行程式的時間點剛好需要開會或處理緊急事件而無法抽身。要是忘記執行爬蟲程式，會造成資料的缺失，這樣會導致日後在實作一些應用時，好比説針對歷史股價進行回測時，只要少了一天的資料，便會造成分析結果不正確，嚴重的話甚至會導致程式出錯。

若有一個機制，能夠讓電腦在我們指定的時間點，自動執行我們寫好的 Python 程式，這樣不但能將搜集資料這件事給完全的自動化了，也能夠避免上述問題發生，減少人爲的失誤。
接下來就跟各位介紹一下排程。簡單來説，排程就是一個能夠電腦在我們指定的時間自動去執行程式的功能，在接下來的章節，我們將學會如何設定排程。

# 5.1.1 Windows 工作排程器

排程在實作上，會因爲作業系統而有很大的差異，這一個章節是講解如何在 Windows 作業系統上實作排程，使用 Mac 或 Linux 的讀者可以直接切換至本教學最後的補充部分。

排程的運作方式，是作業系統會在指定的時間或觸發條件下，透過執行文字命令的方式將程式運作起來，因此我們需要先確保需要被排程的程式碼能夠透過命令列界面成功運作起來。因此在開始之前，先整理一下我們在之前的章節寫的爬蟲程式：


上一課的程式碼：

```python
import requests
from bs4 import BeautifulSoup as BS
import xlwings as xw
import time

res = requests.get("https://stock.wearn.com/cdata.asp?kind=2330")
# 設定目標網頁的編碼
res.encoding = "big5"
html = BS(res.text, "html.parser")
# 用 html 格式解碼 爬下來的檔案
table = html.findAll("table")[0]
trs = table.findAll("tr")
tr = trs[2]
tds = [tr.findAll("td") for tr in trs]
# 第二列
today = tds[2]
data = [float(td.text.replace("\xa0", "").replace(",", "")) for td in today[1:]]


# 開啓 Excel 檔
wb = xw.Book("投資組合.xlsx")
sheet = wb.sheets["2330"]
# 偵測工作表最後一行
last_row = sheet.range("A1").end("down").row
# 將截取到的資料寫入工作表
sheet.range(f"A{last_row+1}").value = time.strftime("%Y/%m/%d")
sheet.range(f"B{last_row+1}").value = price_open
sheet.range(f"C{last_row+1}").value = price_high
sheet.range(f"D{last_row+1}").value = price_low
sheet.range(f"E{last_row+1}").value = price_close
sheet.range(f"F{last_row+1}").value = volume
```

# 5.1.2 把整個投資組合的資料截取到 Excel 内

我們發現到，在 Yahoo 奇摩股市的網頁上，不同股票的資料**都會顯示在 html 結構一樣的網頁上**，而唯一不同的只有**網址**，所以我們其實可以把同樣的程式碼封裝到一個函數内，以便我們將程式模組化：

```python
from bs4 import BeautifulSoup as BS
import requests

# 輸入股票代號，回傳該股票的收盤價
def tw_stock_crawler(stock_id):
    res = requests.get("https://stock.wearn.com/cdata.asp?kind=2330")
    res.encoding = "big5"
    html = BS(res.text, "html.parser")
    table = html.findAll("table")[0] 
    trs = table.findAll("tr")
    tr = trs[2]
    tds = tr.findAll("td")
    data = [float(td.text.replace("\xa0", "").replace(",", "")) for td in tds[1:]]
    # 以字典形式回傳我們的資料
    return {
        "open": data[0],
        "high": data[1],
        "low": data[2],
        "close": data[3],
        "volume": data[4]
    }
```

測試我們寫好的函數：

```python
print(tw_stock_crawler("2412"))
```

{'open': 125.0, 'high': 125.5, 'low': 124.0, 'close': 124.5, 'volume': 12166.0}


利用迴圈呼叫函數，我們就能截取到一整個投資組合的資料：

```python
stocks = ["2330", "2317", "0050", "2412", "2603"] 

for sid in stocks:
    print(tw_stock_crawler(sid))
```

# 5.1.3 迭代每一個工作表

```python
import time
import xlwings as xw

# 開啓活頁簿
wb = xw.Book("投資組合.xlsx")
# 定義所有股票代號
stocks = ["2330", "0050", "2412", "2317", "2603"]

for stock in stocks:
    # 截取相對應的工作表
    sheet = wb.sheets[stock]
    # 顯示開啓工作表
    sheet.activate()
    # 暫停 3 秒鐘
    time.sleep(3)
```

我們整合一下所有的程式碼：

```python
import time
import xlwings as xw

wb = xw.Book("投資組合.xlsx")
date = time.strftime("%Y/%m/%d")
# 定義所有投資組合的股票代號
stocks = ["2330", "0050", "2412", "2317", "2603"]

for stock in stocks:
    # 顯示開啓工作表
    sheet.activate()
    # 截取到該股票代號的收盤價
    data = tw_stock_crawler(stock)
    # 截取相對應的工作表
    sheet = wb.sheets[stock]
    # 偵測該工作表的最後一行行數
    last_row = sheet.range("B1").end("down").row
    # 將資料寫入相對應的欄位
    sheet.range(f"A{last_row+1}").value = date
    sheet.range(f"B{last_row+1}").value = data["open"]
    sheet.range(f"C{last_row+1}").value = data["high"]
    sheet.range(f"D{last_row+1}").value = data["low"]
    sheet.range(f"E{last_row+1}").value = data["close"]
    sheet.range(f"F{last_row+1}").value = data["volume"]
    # 停止兩秒
    time.sleep(2)
```

# 5.1.4 完整版程式碼

In [24]:
import requests
from bs4 import BeautifulSoup as BS
import time
import xlwings as xw

# 輸入股票代號，回傳該股票的收盤價
def tw_stock_crawler(sid):
    res = requests.get(f"https://stock.wearn.com/cdata.asp?kind={sid}")
    res.encoding = "big5"
    html = BS(res.text, "html.parser")
    table = html.findAll("table")[0] 
    trs = table.findAll("tr")
    tr = trs[2]
    tds = tr.findAll("td")
    data = [float(td.text.replace("\xa0", "").replace(",", "")) for td in tds[1:]]
    # 以字典形式回傳我們的資料
    return {
        "open": data[0],
        "high": data[1],
        "low": data[2],
        "close": data[3],
        "volume": data[4]
    }

wb = xw.Book("投資組合.xlsx")
date = time.strftime("%Y/%m/%d")
# 定義所有投資組合的股票代號
stocks = ["2330", "0050", "2412", "2317", "2603"]

for stock in stocks:
    # 顯示開啓工作表
    sheet.activate()
    # 截取到該股票代號的收盤價
    data = tw_stock_crawler(stock)
    # 截取相對應的工作表
    sheet = wb.sheets[stock]
    # 偵測該工作表的最後一行行數
    last_row = sheet.range("B1").end("down").row
    # 將資料寫入相對應的欄位
    sheet.range(f"A{last_row+1}").value = date
    sheet.range(f"B{last_row+1}").value = data["open"]
    sheet.range(f"C{last_row+1}").value = data["high"]
    sheet.range(f"D{last_row+1}").value = data["low"]
    sheet.range(f"E{last_row+1}").value = data["close"]
    sheet.range(f"F{last_row+1}").value = data["volume"]
    time.sleep(2)

# 儲存檔案
wb.save()

In [ ]:
# 若想要在儲存之後關掉 Excel App，可以加上這一行：
wb.app.quit()

# 5.2 設定 Windows 工作排程器

在確認完畢之後，接下來我們需要確認上述程式碼能夠透過執行文字命令的方式順利運作。要做到這件事，我們需要先透過以下流程，將程式碼整理到一個 .py 檔案内：

1. 先在 Jupyter Notebook 的界面上，點選界面右上方的 New 按鈕，選擇 Text File 選項

2. 接下來 Jupyter Notebook 會建立一個 untitled.txt 文字檔案，直接點選該檔案，便能進入編輯界面

3. 在界面内尋找 File 選單，選擇 Rename 選項

4. 將此檔案重新命名為 **tw_stock_crawler.py**

5. 最後再將我們整理好的程式碼複製至此檔案内

6. 另外，別忘了在程式碼内設定投資組合歷史資料.xlsx檔案的完整路徑

7. 在設定完畢投資組合歷史資料.xlsx 檔案的完整路徑之後，接下來我們開啓檔案總管，將 tw_stock_crawler.py 檔案的完整路徑複製下來

8. 接下來開啓 windows 的搜尋功能，尋找命令提示字元：

9. 在命令提示字元，先輸入指令

```bash
python "C:\Users\到 tw_stock_crawler.py 檔案的完整路徑"
```

若上述指令無法執行或者在執行中發生錯誤，可能是因爲電腦内同時存在多個不同版本的 Python，要解決此問題，可以將開頭的 python 指令改成 Anaconda 版本的 python.exe 執行檔案的路徑，我們可以透過以下指令搜尋到該路徑：

```bash
where python
```

Anaconda 的 python.exe 預設的安裝路徑是在：

```bash
C:\ProgramData\Anaconda3\python.exe
```

In [ ]:
!where python

![](14-010.png)

10. 最後我們將 python.exe 執行檔案的路徑以及 tw_stock_crawler.py 檔案的完整路徑結合成以下指令即可：

```bash
C:\ProgramData\Anaconda3\python.exe "C:\Users\到 tw_stock_crawler.py 檔案的完整路徑"
```

![](14-011.png)

注意圖内的指令是以筆者的電腦爲主，請勿直接複製。

在確認該指令可以成功執行後，接下來我們就來設定排程吧。

11. 接下來，我們再次開啓 windows 的搜尋功能，尋找工作排程器

12. 開啓之後，選擇建立基本工作

13. 設定工作名稱，以及備注

14. 接下來設定執行的頻率，由於我們的爬蟲程式是需要搜集台股每一天的交易資訊，因此我們選擇在沒一個交易日（每一周的周一至周五）

15. 由於我們需要截取個股的收盤價，而收盤價必須等到台股收盤之後（每個交易日下午一點半之後），因此我們將時間設定在下午兩點零一分

16. 設定完時間之後，我們選擇啓動程式

17. 請將 python.exe 的路徑注入至程式，再將 tw_stock_crawler.py 檔案的完整路徑輸入至新增引數

18. 確認所有内容

19. 設定完畢之後，建立好的工作應該會出現在工作清單，若要測試，可以點選一下執行


# 5.3 補充：設定 Mac OS 的 crontab

不同於 windows 作業系統有一個圖形化的工作排程器可以輔助使用者設定排程。Mac OS 以及 Linux 作業系統則是要求開發者將要執行的指令以及頻率設定寫入一個叫做 crontab 的特殊設定檔案内。

1. 我們先開啓 Terminal，或是任何一個終端機應用程式：

![](14-026.png)


2. 接下來我們在終端機應用程式執行：

```bash
crontab -e
```

![](14-028.png)

便可透過終端機應用程式内建的 vi 編輯器開啓 crontab 設定儅。

3. 此時我們的設定檔案應該是一個空的檔案，

一般 crontab 的設定方式為下：

```bash
 ┌─────────────────────── 分鐘
 │   ┌─────────────────── 小時   
 │   │    ┌────────────── 日    
 │   │    │   ┌────────── 月    
 │   │    │   │   ┌────── 星期幾 
 │   │    │   │   │
MIN HOUR DOM MON DOW CMD
```

前面五個設定值為設定頻率的特殊語法，分別代表不同時間單位的頻率設定，而最後一個設定值 CMD 則是代表需要被執行的檔案。

針對每一個頻率設定值背後代表的意義，請參考以下表格：

|欄位|說明|可設定的值|
|-----|:----:|------:|
|MIN  | 分鐘 | 0 到 59|
|HOUR | 小時 | 0 到 23|
|DOM  | 日   | 1 到 31|
|MON  | 月份 | 1 到 12，此欄位亦可用英文簡稱取代，例如一月也可以寫 Jan。|
|DOW  |星期幾|	0（週日）到 6（週六），7 也代表週日。此欄位亦可用英文簡稱取代，例如週日也可以寫 Sun。|

另外，crontab 亦可使用一些特殊字元，每個特殊字元都有不同的意義與適用情況：


|特殊字符|說明 |
|-----|:----:|
|*（星號）| 代表接受任意時刻，例： * * * * * 代表每天每小時每分鐘執行指令 |
 ,（逗號）| 代表分隔時段，例： * 9, 12, 15 * * * 代表每天九點、十二點、下午三點執行指令 |
|-（減號）| 代表一段時間範圍，例： * 9-18 * * * 代表從早上 9 點到下午 6 點凡是整點就執行指令 |
|/n（斜線）| n 代表數字，表示每個 n 單位間隔，例： */10 * * * * 代表每天每小時每隔 10 分鐘執行指令 |

因此，若我們希望程式是在每一天的下午兩點零一分啓動，在 crontab 的設定方式則為：

```bash
1 14 * * * python /path/to/tw_stock_crawler.py
```

上述的 `/path/to/tw_stock_crawler.py` 代表到 `tw_stock_crawler.py` 檔案的完整路徑。

![](14-029.png)

在編輯完畢之後，只需要按下退出鍵（ESC），再輸入 `:wq` 即可退出 vi 編輯器。Mac OS 與 Linux 的排程設定也就完成了。 
